In [1]:
import os

In [2]:
%pwd

'/home/kalema/Projects/Kidney-Disease-Classification-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/kalema/Projects/Kidney-Disease-Classification-MLflow-DVC'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kalema3502/Kidney-Disease-Classification-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kalema3502"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fb3845efcc3b2e46a4157b1d2c977a21e02dd16e"

In [13]:
import tensorflow as tf
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
class ConfigurationManager:
    """Class for managing configuration files and preparing base models.
    
    This class handles the loading of configuration files and parameters,
    as well as the creation of directories necessary for preparing base models.
    
    Attributes:
        config_filepath (str, optional): The filepath of the configuration file. Defaults to CONFIG_FILE_PATH.
        params_filepath (str, optional): The filepath of the parameters file. Defaults to PARAMS_FILE_PATH.
    
    Methods:
        get_prepare_base_model_config(): Retrieves the configuration for preparing base models.
    """

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """Initializes the ConfigurationManager.

        Args:
            config_filepath (str, optional): The filepath of the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): The filepath of the parameters file. Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/kalema3502/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return evaluation_config

In [17]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def valid_generator(self):
        """
        Prepares data generators for training and validation.

        This method prepares data generators for training and validation using the specified parameters
        in the training configuration. It applies data augmentation techniques if enabled.
        """
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.validation_data = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.Model.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.validation_data)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # No signature
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")